In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import sys
import platform
# from xgboost import XGBClassifier
from xgboost.sklearn import XGBClassifier
from scipy import stats
import datetime as dt
# from scipy.stats import ks_2samp
import pandas as pd
import numpy as np
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import multiprocessing
import os
os.environ["KMP_DUPLICATE_LIB_OK"]='True'

In [2]:
print(sys.version)

3.5.2 |Anaconda custom (x86_64)| (default, Jul  2 2016, 17:52:12) 
[GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]


In [3]:
print(platform.python_version())

3.5.2


In [4]:
data = pd.read_csv("vars.csv")

In [5]:
data['Date'] = pd.to_datetime(data['Date'])
data.dtypes

Date                          datetime64[ns]
Recnum                                 int64
Fraud                                  int64
cardnum_1                              int64
merchnum_oncard_unique_1               int64
merchzip_oncard_unique_1               int64
merchnum_1                             int64
cardnum_onmerch_unique_1               int64
merchzip_onmerch_unique_1              int64
cardnum_3                              int64
merchnum_oncard_unique_3               int64
merchzip_oncard_unique_3               int64
merchnum_3                             int64
cardnum_onmerch_unique_3               int64
merchzip_onmerch_unique_3              int64
cardnum_7                              int64
merchnum_oncard_unique_7               int64
merchzip_oncard_unique_7               int64
merchnum_7                             int64
cardnum_onmerch_unique_7               int64
merchzip_onmerch_unique_7              int64
cardnum_30                             int64
merchnum_o

In [6]:
data.describe()

,Recnum,Fraud,cardnum_1,merchnum_oncard_unique_1,merchzip_oncard_unique_1,merchnum_1,cardnum_onmerch_unique_1,merchzip_onmerch_unique_1,cardnum_3,merchnum_oncard_unique_3,...,cardnum_30,merchnum_oncard_unique_30,merchzip_oncard_unique_30,merchnum_30,cardnum_onmerch_unique_30,merchzip_onmerch_unique_30,cardnum_lastseen,cardnum_zscore,merchnum_lastseen,merchnum_zscore
count,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,...,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000
mean,48377.000000,0.010945,2.470383,1.333736,1.255868,7.001085,3.375844,1.072349,4.026242,2.140058,...,19.890587,8.782126,7.922204,150.646884,24.785485,4.129267,355.269759,0.020125,306.037560,0.015150
std,27930.329635,0.104047,5.991622,0.816828,0.806885,18.966278,5.865220,0.801873,8.497033,1.791928,...,30.399858,8.039992,7.312214,365.274698,40.559014,17.330093,48.289103,1.011652,125.066024,0.947895
min,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,-2.458853,0.000000,-3.423952
25%,24189.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,3.000000,3.000000,2.000000,1.000000,1.000000,361.000000,-0.578009,345.000000,-0.499299
50%,48377.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,...,11.000000,7.000000,6.000000,7.000000,5.000000,1.000000,365.000000,-0.319743,365.000000,-0.154319
75%,72565.000000,0.000000,2.000000,1.000000,1.000000,3.000000,2.000000,1.000000,4.000000,3.000000,...,23.000000,11.000000,10.000000,54.000000,27.000000,1.000000,366.000000,0.212071,366.000000,0.109183
max,96753.000000,1.000000,146.000000,24.000000,22.000000,260.000000,45.000000,19.000000,177.000000,37.000000,...,425.000000,81.000000,73.000000,1786.000000,222.000000,137.000000,366.000000,16.881559,366.000000,46.531249


In [7]:
var_columns = data.columns.drop(['Date', 'Recnum', 'Fraud'])
var_columns.astype(list)

Index(['cardnum_1', 'merchnum_oncard_unique_1', 'merchzip_oncard_unique_1',
       'merchnum_1', 'cardnum_onmerch_unique_1', 'merchzip_onmerch_unique_1',
       'cardnum_3', 'merchnum_oncard_unique_3', 'merchzip_oncard_unique_3',
       'merchnum_3', 'cardnum_onmerch_unique_3', 'merchzip_onmerch_unique_3',
       'cardnum_7', 'merchnum_oncard_unique_7', 'merchzip_oncard_unique_7',
       'merchnum_7', 'cardnum_onmerch_unique_7', 'merchzip_onmerch_unique_7',
       'cardnum_30', 'merchnum_oncard_unique_30', 'merchzip_oncard_unique_30',
       'merchnum_30', 'cardnum_onmerch_unique_30',
       'merchzip_onmerch_unique_30', 'cardnum_lastseen', 'cardnum_zscore',
       'merchnum_lastseen', 'merchnum_zscore'],
      dtype='object')

In [8]:
# make a "perfect" input or completely random inputs to check model resutls

# perfect input:
# data['perfect'] = data['Fraud']

# random inputs:
# nrecords = len(data)
# orig_columns = data.columns
# ncolumns = len(orig_columns)
# datarand = pd.DataFrame(np.random.randint(0,5, size = (nrecords, ncolumns)), columns = orig_columns)
# datarand["Date"] = pd.to_datetime(data['Date'])
# datarand['Recnum'] = data['Recnum']
# datarand['Fraud'] = data['Fraud']
# data = datarand
# data.head()

In [9]:
data.fillna(0)
data.loc[:,var_columns] = data[var_columns].clip(upper=10)
data.describe()

,Recnum,Fraud,cardnum_1,merchnum_oncard_unique_1,merchzip_oncard_unique_1,merchnum_1,cardnum_onmerch_unique_1,merchzip_onmerch_unique_1,cardnum_3,merchnum_oncard_unique_3,...,cardnum_30,merchnum_oncard_unique_30,merchzip_oncard_unique_30,merchnum_30,cardnum_onmerch_unique_30,merchzip_onmerch_unique_30,cardnum_lastseen,cardnum_zscore,merchnum_lastseen,merchnum_zscore
count,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,...,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000,96753.000000
mean,48377.000000,0.010945,1.955123,1.330584,1.252902,2.832057,2.506372,1.069548,3.054345,2.115004,...,7.794373,6.425175,6.034490,6.130011,5.537472,1.302606,9.829979,0.019670,8.646791,0.011969
std,27930.329635,0.104047,1.994103,0.766657,0.759230,3.117321,2.847631,0.763007,2.619731,1.596826,...,3.005480,3.204861,3.239046,3.939481,4.013054,1.657384,1.292794,1.006230,3.420597,0.873380
min,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,-2.458853,0.000000,-3.423952
25%,24189.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,5.000000,3.000000,3.000000,2.000000,1.000000,1.000000,10.000000,-0.578009,10.000000,-0.499299
50%,48377.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,...,10.000000,7.000000,6.000000,7.000000,5.000000,1.000000,10.000000,-0.319743,10.000000,-0.154319
75%,72565.000000,0.000000,2.000000,1.000000,1.000000,3.000000,2.000000,1.000000,4.000000,3.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,10.000000,0.212071,10.000000,0.109183
max,96753.000000,1.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [2]:
# variable reduction, sorted by univariate KS. strongest v's at bottom

data.drop('cardnum_onmerch_unique_1', axis = 1, inplace = True)
data.drop('cardnum_onmerch_unique_30', axis = 1, inplace = True)
data.drop('merchzip_onmerch_unique_30', axis = 1, inplace = True)
data.drop('merchzip_onmerch_unique_7', axis = 1, inplace = True)
data.drop('merchzip_onmerch_unique_3', axis = 1, inplace = True)
data.drop('merchzip_onmerch_unique_1', axis = 1, inplace = True)
data.drop('cardnum_onmerch_unique_3', axis = 1, inplace = True)
data.drop('cardnum_onmerch_unique_7', axis = 1, inplace = True)
data.drop('merchzip_oncard_unique_30', axis = 1, inplace = True)
data.drop('merchnum_oncard_unique_1', axis = 1, inplace = True)
data.drop('merchnum_oncard_unique_30', axis = 1, inplace = True)
data.drop('merchzip_oncard_unique_1', axis = 1, inplace = True)
data.drop('merchzip_oncard_unique_3', axis = 1, inplace = True)
data.drop('merchnum_oncard_unique_3', axis = 1, inplace = True)
data.drop('merchzip_oncard_unique_7', axis = 1, inplace = True)
data.drop('merchnum_oncard_unique_7', axis = 1, inplace = True)
data.drop('merchnum_1', axis = 1, inplace = True)
data.drop('merchnum_30', axis = 1, inplace = True)
data.drop('merchnum_lastseen', axis = 1, inplace = True)
# data.drop('merchnum_7', axis = 1, inplace = True)
# data.drop('merchnum_3', axis = 1, inplace = True)
# data.drop('cardnum_30', axis = 1, inplace = True)
# data.drop('cardnum_lastseen', axis = 1, inplace = True)
# data.drop('cardnum_1', axis = 1, inplace = True)
# data.drop('cardnum_3', axis = 1, inplace = True)
# data.drop('cardnum_7', axis = 1, inplace = True)
# data.drop('cardnum_zscore', axis = 1, inplace = True)
# data.drop('merchnum_zscore', axis = 1, inplace = True)

NameError: name 'data' is not defined

In [11]:
data.shape

(96753, 21)

In [12]:
# remove the first 10 days for training
data = data[data['Date'] > '2010-01-10']

In [13]:
data.shape

(94893, 21)

In [14]:
trntst = data[data['Date'] < '2010-11-01']
oot = data[data['Date'] >= '2010-11-01']
trntst.shape

(82439, 21)

In [15]:
oot.shape

(12454, 21)

In [16]:
trntst.head(10)

,Date,Recnum,Fraud,cardnum_1,merchzip_oncard_unique_1,merchnum_1,cardnum_3,merchnum_oncard_unique_3,merchzip_oncard_unique_3,merchnum_3,...,merchnum_oncard_unique_7,merchzip_oncard_unique_7,merchnum_7,cardnum_30,merchnum_oncard_unique_30,merchnum_30,cardnum_lastseen,cardnum_zscore,merchnum_lastseen,merchnum_zscore
1860,2010-01-11,1861,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,2.0,1.0,2.0,10.0,0.707107,10.0,0.707107
1861,2010-01-11,1862,0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,...,3.0,3.0,2.0,4.0,3.0,2.0,10.0,1.358259,10.0,0.707107
1862,2010-01-11,1863,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,3.0,3.0,1.0,3.0,3.0,2.0,10.0,0.043525,10.0,-0.707107
1863,2010-01-11,1864,0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,...,1.0,1.0,10.0,4.0,2.0,10.0,10.0,-1.500000,10.0,-0.766585
1864,2010-01-11,1865,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,2.0,2.0,1.0,10.0,-0.707107,0.0,0.000000
1865,2010-01-11,1866,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,1.0,3.0,3.0,1.0,10.0,-0.755591,0.0,0.000000
1866,2010-01-11,1867,0,1.0,1.0,1.0,5.0,4.0,4.0,1.0,...,5.0,6.0,1.0,7.0,5.0,1.0,10.0,-0.195600,0.0,0.000000
1867,2010-01-11,1868,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,2.0,2.0,1.0,10.0,0.707107,0.0,0.000000
1868,2010-01-11,1869,0,1.0,1.0,1.0,2.0,2.0,2.0,6.0,...,4.0,4.0,10.0,5.0,4.0,10.0,10.0,-1.203863,10.0,-0.598728
1869,2010-01-11,1870,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,1.0,3.0,3.0,1.0,10.0,1.042288,0.0,0.000000


In [17]:
uncolumns = ['Date', 'Fraud', 'Recnum']
X = trntst.drop(uncolumns, axis=1)
X.head()

,cardnum_1,merchzip_oncard_unique_1,merchnum_1,cardnum_3,merchnum_oncard_unique_3,merchzip_oncard_unique_3,merchnum_3,cardnum_7,merchnum_oncard_unique_7,merchzip_oncard_unique_7,merchnum_7,cardnum_30,merchnum_oncard_unique_30,merchnum_30,cardnum_lastseen,cardnum_zscore,merchnum_lastseen,merchnum_zscore
1860,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,10.0,0.707107,10.0,0.707107
1861,1.0,1.0,1.0,3.0,3.0,3.0,1.0,4.0,3.0,3.0,2.0,4.0,3.0,2.0,10.0,1.358259,10.0,0.707107
1862,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,1.0,3.0,3.0,2.0,10.0,0.043525,10.0,-0.707107
1863,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,1.0,1.0,10.0,4.0,2.0,10.0,10.0,-1.500000,10.0,-0.766585
1864,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,10.0,-0.707107,0.0,0.000000


In [18]:
X.shape

(82439, 18)

In [19]:
uncolumns = ['Date', 'Fraud', 'Recnum']
X_oot = oot.drop(uncolumns, axis=1)

In [20]:
Y_oot = pd.DataFrame(oot['Fraud'])

In [21]:
Y = pd.DataFrame(trntst['Fraud'])

In [22]:
sdev = X.std()
avg = X.mean()
X = (X - avg) / sdev

In [23]:
X_oot.describe()

,cardnum_1,merchzip_oncard_unique_1,merchnum_1,cardnum_3,merchnum_oncard_unique_3,merchzip_oncard_unique_3,merchnum_3,cardnum_7,merchnum_oncard_unique_7,merchzip_oncard_unique_7,merchnum_7,cardnum_30,merchnum_oncard_unique_30,merchnum_30,cardnum_lastseen,cardnum_zscore,merchnum_lastseen,merchnum_zscore
count,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000,12454.000000
mean,1.845752,1.203228,2.416091,2.803758,2.030352,1.853059,3.346636,4.100530,3.098041,2.826481,4.241850,7.419464,6.271559,5.865826,9.981532,-0.048707,9.135458,-0.022117
std,1.865137,0.715244,2.706794,2.466224,1.534227,1.475709,3.351280,3.019199,2.316437,2.195606,3.731832,3.116269,3.205702,3.939586,0.429364,0.939419,2.809691,0.843088
min,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,-2.046653,0.000000,-2.945241
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,5.000000,3.000000,2.000000,10.000000,-0.594129,10.000000,-0.518187
50%,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,3.000000,2.000000,2.000000,2.000000,9.000000,6.000000,6.000000,10.000000,-0.379179,10.000000,-0.140627
75%,2.000000,1.000000,2.000000,3.000000,2.000000,2.000000,5.000000,6.000000,4.000000,4.000000,9.000000,10.000000,10.000000,10.000000,10.000000,0.097119,10.000000,0.079555
max,10.000000,9.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [24]:
sdev = X_oot.std()
avg = X_oot.mean()
X_oot = (X_oot - avg) / (sdev + .01)

In [25]:
niter = 0
nitermax = 10
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
FDR3.head()

,trn,tst,oot
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [26]:
%%time
X_oot_orig = X_oot.copy()

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X, Y, test_size = .3)

#     logistic, RF, NN models here

#     model = LogisticRegression()

#     model = RandomForestClassifier(bootstrap = True, 
#                                         max_depth = 20, 
#                                         max_features = 20, 
#                                         min_samples_leaf = 5, 
#                                         min_samples_split = 10,
#                                         n_estimators = 2000,
#                                         oob_score = True,
#                                         n_jobs = -1)

#     model = MLPClassifier(hidden_layer_sizes = (3),
#                                activation = 'logistic',
#                                max_iter = 200,
#                                alpha= .0001,
#                                solver = 'sgd',
#                                verbose = 10,
#                                tol = .00001,
#                                learning_rate = 'adaptive',
#                                learning_rate_init = .2)

#     X_trn_array = X_trn.as_matrix()
#     X_tst_array = X_tst.as_matrix()
#     X_oot_array = X_oot.as_matrix()
#     Y_trn_array = Y_trn.as_matrix()
#     Y_tst_array = Y_tst.as_matrix()
#     Y_oot_array = Y_oot.as_matrix()
    
    X_trn_array = X_trn.values
    X_tst_array = X_tst.values
    X_oot_array = X_oot.values
    Y_trn_array = Y_trn.values
    Y_tst_array = Y_tst.values
    Y_oot_array = Y_oot.values
    
    xg_trn = xgb.DMatrix(X_trn_array, label = Y_trn_array)
    xg_tst = xgb.DMatrix(X_tst_array, label = Y_tst_array)
    xg_oot = xgb.DMatrix(X_oot_array, label = Y_oot_array)
    params = {'max_depth':5, 'eta':.005, 'silent':0, 'objective':'binary:logistic'}
    num_round = 1200
    model = xgb.train(params, xg_trn, num_round)


    X_oot = X_oot_orig.copy()
    X_trn_eval = X_trn.copy()
    X_tst_eval = X_tst.copy()
    X_oot_eval = X_oot.copy()
       
#     model.fit(X_trn, Y_trn.values.ravel())   

#     predictions = model.predict_proba(X_trn)[:,1]
    predictions = model.predict(xg_trn)
    X_trn_eval['predicted']=predictions
    X_trn_eval['Fraud'] = Y_trn['Fraud']
    topRows = int(round(X_trn_eval.shape[0]*0.03))
    temp = X_trn_eval.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn_eval.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_tst)[:,1]
    predictions = model.predict(xg_tst)
    X_tst_eval['predicted']=predictions
    X_tst_eval['Fraud'] = Y_tst['Fraud']
    topRows = int(round(X_tst_eval.shape[0]*0.03))
    temp = X_tst_eval.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst_eval.loc[:,'Fraud'])

#     predictions = model.predict_proba(X_oot)[:,1]
    predictions = model.predict(xg_oot)
    X_oot_eval['predicted']=predictions
    X_oot_eval['Fraud'] = Y_oot['Fraud']
    topRows = int(round(X_oot_eval.shape[0]*0.03))
    temp = X_oot_eval.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'Fraud']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot_eval.loc[:,'Fraud'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])

FDR3.head(10)

0 0.752827140549 0.679841897233 0.486033519553
1 0.765517241379 0.678082191781 0.458100558659
2 0.751655629139 0.649253731343 0.491620111732
3 0.757525083612 0.653284671533 0.458100558659
4 0.765089722675 0.664092664093 0.486033519553
5 0.769867549669 0.634328358209 0.430167597765
6 0.768718801997 0.638376383764 0.441340782123
7 0.76660988075 0.645614035088 0.474860335196
8 0.770247933884 0.674157303371 0.469273743017
9 0.792604501608 0.58 0.469273743017
CPU times: user 31min 51s, sys: 20.8 s, total: 32min 12s
Wall time: 8min 54s


In [27]:
FDR3.describe()

,trn,tst,oot
count,10.000000,10.000000,10.000000
mean,0.766066,0.649703,0.466480
std,0.011550,0.029386,0.019926
min,0.751656,0.580000,0.430168
25%,0.759416,0.640186,0.458101
50%,0.766064,0.651269,0.469274
75%,0.769580,0.671641,0.483240
max,0.792605,0.679842,0.491620


In [28]:
FDR3.to_csv('parms_temp.csv', index = False)
X.shape

(82439, 18)